### Shedding
In emergency conditions, it may be possible / necessary for a grid operator to disconnect certain loads, generators, or other components in order to prevent a larger blackout. This notebook explores how this can be achieved in Grid2OP. 

By default shedding is disabled in all environments, to provide the keyword argument allow_shedding when initializing the environment.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import grid2op
from lightsim2grid import LightSimBackend
from grid2op.Parameters import Parameters

from grid2op.PlotGrid import PlotMatplot
from pathlib import Path

data_path = Path.cwd() / "grid2op" / "data"
p = Parameters()
p.MAX_SUB_CHANGED = 5
env = grid2op.make(data_path / "rte_case5_example", param=p, allow_shedding=True)
plotter = PlotMatplot(env.observation_space, load_name=True, gen_name=True, dpi=150)
print(f"Loads: {env.n_load}, Generators: {env.n_gen}, Storage: {env.n_storage}, Allow Shedding: {env.allow_shedding}")

In [ ]:
# Disconnect the load at substation 4
load_lookup = {name:i for i,name in enumerate(env.name_load)}
gen_lookup = {name:i for i,name in enumerate(env.name_gen)}
act = env.action_space({"set_bus":[(env.load_pos_topo_vect[load_lookup["load_4_2"]], -1),
                                   (env.load_pos_topo_vect[load_lookup["load_3_1"]], -1)]})
# act = env.action_space({"set_bus":[(env.gen_pos_topo_vect[gen_lookup["gen_0_0"]], -1)]})
print(act)
env.set_id("00")
init_obs = env.reset()
obs, reward, done, info = env.step(act)
plotter.plot_obs(obs, figure=plt.figure(figsize=(8,5)))
plt.show()

In [ ]:
import pandapower as pp
network = env.backend._grid.deepcopy()
display(network.res_line.loc[:, ["p_from_mw", "p_to_mw", "q_from_mvar", "q_to_mvar"]])
pp.runpp(network,
         check_connectivity=False,
         init="dc",
         lightsim2grid=False,
         max_iteration=10,
         distributed_slack=False,
)
display(network.res_line.loc[:, ["p_from_mw", "p_to_mw", "q_from_mvar", "q_to_mvar"]])